## Scenario: **Exploring Potential Oil and Gas Drilling Sites Near San Francisco's Industrial Development Zone (IDZ)**
*Notebook 5 of 5*


With the increasing global demand for energy, San Francisco's outskirts have been identified as potential regions for oil and gas exploration. San Francisco's Industrial Development Division is keen to discover and capitalize on new oil and gas reservoirs. However, to ensure minimal environmental disruption and efficient transportation of resources, these drilling sites must be near the newly proposed Industrial Development Zone (IDZ).

### Objective:

The aim is to identify potential drilling sites within a designated proximity to the IDZ, which is represented as a hexagonal boundary. These potential sites, once identified, will undergo geological and environmental impact assessments.

### Procedure:

- **Data Collection**: Geospatial data for potential drilling sites, given by their latitudes and longitudes, is gathered. The IDZ's location is represented using the H3 geospatial indexing system, centered around a specified set of coordinates.

- **Buffering**: To ensure the potential sites are in feasible regions for exploration, a buffer zone is delineated around the IDZ. Any site within this buffer can be connected to the IDZ with efficient logistics.

- **Spatial Joining**: Through spatial operations, we pinpoint which potential drilling sites are located within this buffer.

- **Spatial Autocorrelation Analysis**: To efficiently select drilling sites, a spatial autocorrelation analysis is employed to detect clusters of sites that have similar geological features, indicating a larger reservoir. This info aids in strategic planning, as larger reservoirs can cater to higher energy demands.

- **Visualization**: The culmination of the analysis is visualized. The map illustrates the IDZ, the buffer zone, all potential drilling sites, and those within the buffer. Furthermore, clusters based on geological similarity are emphasized, assisting the Industrial Development Division in their subsequent actions.

By adhering to this methodology, San Francisco's outskirts can be transformed into a thriving hub for oil and gas exploration, aligning with environmental considerations and efficient logistics.


In [2]:
pip install geopandas pandas h3 matplotlib pysal splot shapely pyproj

Note: you may need to restart the kernel to use updated packages.



### Library Imports:
The scenario revolves around geospatial operations. The imported libraries, such as `geopandas`, `pandas`, `pysal`, and `splot`, equip us with the tools needed for a range of operations. These go from basic geospatial data manipulation to advanced spatial autocorrelation analysis.

### WKT Parsing:
The well-known text (WKT) representation is a ubiquitous standard in the geospatial domain. It is used for representing geometric objects in a readable text format. In this context, we use it to parse and derive the geometry of a point. This point can symbolize specific city locations or even represent locations of oil/gas wells.

### Coordinate Transformation:
Different geospatial datasets might be represented in different coordinate reference systems. Transforming coordinates is pivotal to ensure alignment across these datasets. This step is especially crucial when preparing data for specific visualizations or analyses or when integrating with web-based mapping tools.


In [3]:
import geopandas as gpd
import pandas as pd
import h3
import matplotlib.pyplot as plt
from pysal.explore import esda
from splot.esda import plot_local_autocorrelation
from shapely.geometry import Point, shape
from shapely import wkb, wkt
from pyproj import Transformer

# Parsing WKT to a Shapely Geometry
wkt_string = "POINT(30 10)"
geometry_from_wkt = wkt.loads(wkt_string)

# Change the CRS of the Shapely Geometry from EPSG:4326 to EPSG:3857
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
x, y = geometry_from_wkt.xy
x_transformed, y_transformed = transformer.transform(x[0], y[0])
transformed_point = Point(x_transformed, y_transformed)



## Relevance to the Scenario:

### Mock Dataset Creation:
This part creates a mock dataset representing existing wells with their geographical locations and production rates. The wells could be potential freshwater sources for the city's Urban Development Zone (UDZ). The production rate might be used later on to identify high-yielding wells.

The provided code sets up a foundation for the scenario by defining the region of interest (the hexagonal UDZ) and a sample dataset of wells. Further operations, such as buffering, spatial joining, and analysis, can then be performed to meet the objectives of the scenario.


In [ ]:
# Using H3 for hexagonal tesselation
lat, lon = 37.7749, -122.4194  # San Francisco coordinates
h3_index = h3.geo_to_h3(lat, lon, resolution=7)
boundary = h3.h3_to_geo_boundary(h3_index)
hexagon = gpd.GeoDataFrame([1], geometry=[shape({"type": "Polygon", "coordinates": [boundary]})])

# Sample Data (for the sake of example, use random values)
data = {
    'longitude': [30, 31, 32, 33, 34],
    'latitude': [10, 11, 12, 13, 14],
    'production_rate': [100, 150, 200, 250, 300]
}
df = pd.DataFrame(data)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))




## Overview

The code in focus assists in identifying spatial patterns in well productivity, specifically how wells correlate with one another and the UDZ (Urban Development Zone). Visualization of these patterns can guide efficient planning and decision-making for the city's freshwater needs.

### K-nearest Neighbors (KNN) and Spatial Weights

**What it does:**
- Determines the number of neighbors to consider for each data point in the GeoDataFrame `gdf`. This is determined by the minimum of 5 or one less than the total number of points.
- Calculates the k-nearest neighbors spatial weights matrix for the dataset.

**Relevance to the Scenario:**
The spatial weights matrix establishes how each observation in the dataset is spatially related to every other observation. This matrix is vital in spatial analysis, helping identify and quantify patterns or relationships between data points.

### Global Moran's I Statistical Test

**What it does:**
- Performs a Global Moran's I statistical test to gauge spatial autocorrelation. This is based on the production rates of the wells, using the previously defined spatial weights.

**Relevance to the Scenario:**
The analysis helps determine if there's a discernible spatial pattern in the production rates of the wells. For instance, the city can ascertain whether high-producing wells cluster near other high-producing wells or if the distribution is random. This knowledge aids decision-making related to water supply distribution or potential drilling projects.

### Visualization of Well Data and UDZ

**What it does:**
- Initializes a map for visualization.
- Illustrates the wells on the map using colors that represent their production rates.
- Superimposes the boundary of the hexagonal tessellation (UDZ) on the map.

**Relevance to the Scenario:**
Visual data representations provide an immediate understanding of the spatial distribution of well productivity and its relationship with the UDZ. This visual context allows planners to quickly identify the locations of high-yielding wells concerning the UDZ.

### Local Moran's I and Visualization

**What it does:**
- Calculates the Local Moran's I statistic for each well to pinpoint local clusters of similar or differing values.
- Showcases these local patterns on a map.

**Relevance to the Scenario:**
Visualizing the Local Moran's I values allows urban planners to pinpoint clusters of high-yielding wells or possible outliers. Recognizing these clusters is essential for infrastructure planning and ensures efficient water supply distribution to the UDZ.

```


In [ ]:
# Compute spatial weights
k = min(5, len(gdf) - 1)
weights = weights.KNN.from_dataframe(gdf, k=k)

# Spatial Autocorrelation Analysis
moran = esda.Moran(gdf['production_rate'], weights)

# Visualization
fig, ax = plt.subplots(figsize=(10, 10))
gdf.plot(column='production_rate', cmap='coolwarm', legend=True, ax=ax)
hexagon.boundary.plot(ax=ax, color='black', linewidth=1)
ax.set_title('Well Production with Hexagonal Tessellation')
plt.show()

# Visualize Local Moran's I
local_moran = esda.Moran_Local(gdf['production_rate'], weights)
plot_local_autocorrelation(local_moran, gdf, 'production_rate')
plt.show()

## Relevance to the Scenario:

### Spatial Joining:
Spatial joining is a process of combining datasets based on their spatial relationship. In this context, we're identifying which of the existing wells are located within the buffered region around the UDZ. By doing this, the city can pinpoint specific wells that are optimal for providing water to the new infrastructures within or near the UDZ.

### Visualization:
Visualization is a powerful tool for urban planning. The map created here depicts the UDZ, the extended (buffered) area around it, all the existing wells, and specifically the wells that fall within the buffered zone. By displaying these elements, city planners can gain a clear spatial understanding of where the most accessible and potential wells are in relation to the UDZ, which aids in decision-making and planning.

### Summary:
In essence, this code segment aids in identifying and visualizing potential freshwater sources (wells) around San Francisco's UDZ, which is essential for ensuring a consistent water supply for the upcoming infrastructures.


In [ ]:
# Create a buffer around the hexagon boundary
# For this example, we'll buffer by a distance of 0.05 degrees (roughly 5km at the equator)
buffered_hexagon = hexagon.buffer(0.05)

# Perform a spatial join to identify wells that fall within the buffered hexagon
# op='within' checks if the wells (points) are within the buffered hexagon
joined_gdf = gpd.sjoin(gdf, buffered_hexagon, how="inner", op="within")

# Visualization with buffering and the wells within the buffered region
fig, ax = plt.subplots(figsize=(10, 10))
buffered_hexagon.boundary.plot(ax=ax, color='blue', linewidth=1, label='Buffered Hexagon')
hexagon.boundary.plot(ax=ax, color='black', linewidth=1, label='Original Hexagon')
gdf.plot(ax=ax, marker='o', color='red', markersize=5, label='All Wells')
joined_gdf.plot(ax=ax, marker='x', color='green', markersize=100, label='Wells within Buffered Hexagon')
plt.legend(loc="upper left")
plt.show()
